# Bias correction process

This example shows how to call the bias-correction service based on the Kernel Density Distribution Mapping.
We first connect to the WPS server and get some information on the service, identified by id `kddm_bc`. 

In [1]:
from owslib.wps import WebProcessingService

url = "https://pavics.ouranos.ca/twitcher/ows/proxy/flyingpigeon/wps"
wps = WebProcessingService(url=url)
proc = wps.describeprocess("kddm_bc")
print(proc.abstract)

Bias correction method using Kernel Density Distribution Mapping (KDDM).


For the next step, we'll create small synthetic files and run the process. 

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

obs_time_index = pd.date_range(start="2000-01-01", end="2000-12-31", freq="D")
obs = xr.DataArray(
    np.arange(len(obs_time_index)), coords={"time": obs_time_index}, dims="time"
)
ref = obs + 1

fut_time_index = pd.date_range(start="2050-01-01", end="2050-12-31", freq="D")
fut = xr.DataArray(
    np.arange(len(fut_time_index)) + 10, coords={"time": fut_time_index}, dims="time"
)

fn = {"obs": "/tmp/obs.nc", "ref": "/tmp/ref.nc", "fut": "/tmp/fut.nc"}

obs.to_netcdf(fn["obs"])
ref.to_netcdf(fn["ref"])
fut.to_netcdf(fn["fut"])

resp = wps.execute(
    "kddm_bc", inputs=[("obs", fn["obs"]), ("ref", fn["ref"]), ("fut", fn["fut"])]
)

In [3]:
print(resp.status)

ProcessAccepted
